# Lesson 05B - Using GridSearchCV


Besides grid search and randomized search, there is a new approach called Bayesian Opptimization. More info can be found [here](https://github.com/shibuiwilliam/keras_gpyopt), [here](https://github.com/keras-team/keras/issues/2477), [great example here](https://github.com/maxpumperla/hyperas) and a [list of libraries](https://medium.com/@mikkokotila/a-comprehensive-list-of-hyperparameter-optimization-tuning-solutions-88e067f19d9)

In [19]:
from keras.datasets import imdb
import pandas as pd
import numpy as np

from keras.preprocessing import sequence

**Prepare the data**

In [20]:
max_features = 5000

In [21]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) # instead on nb_words=

I can use another argument here, which is 'skip_top=', and that to ignore X top most frequent words

In [22]:
maxlen = 400

We define maximum length of 400. It means that posts shoeter than 400 will be filled with 0s, and longer posts are cut.

In [23]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

The first two arrays were padded, and the third one was cut.

**Now defining the model**

As opposed to the original file, I an creating the model in a function.

Also gathering all the imports from the original version (05) in the following cell

In [24]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import Convolution1D
from keras.layers import Conv1D
from keras.layers import GlobalMaxPooling1D
from keras.layers import Dense, Dropout, Activation

Keras models can be used in scikit-learn by wrapping them with the **KerasClassifier** or **KerasRegressor** class.

To use these wrappers you must define a function that creates and returns your Keras sequential model, then pass this function to the build_fn argument when constructing the KerasClassifier class.

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [26]:
def create_model(dropout=0.2):
    # create model
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=50, input_length=maxlen))
    model.add(SpatialDropout1D(dropout))
    model.add(Convolution1D(
                            filters=250, # instead of nb_filter=
                            kernel_size=3, # instead of filter_length=
                            padding='valid', # Instead of border_mode=
                            activation='relu',
                            strides=1)) #instead of subsample_length=
    model.add(GlobalMaxPooling1D())
    model.add(Dense(250)) # I define the output of the dense (vector length after the layer). It remains 250
    model.add(Dropout(dropout))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [27]:

# model.add(
#     Convolution1D(
#         filters=250, # instead of nb_filter=
#         kernel_size=3, # instead of filter_length=
#         padding='valid', # Instead of border_mode=
#         activation='relu',
#         strides=1, #instead of subsample_length=
#     )
# )

Within the KerasClassifier I include the original fit agruments (batch_size, epochs, verbose), unless I want to grid search them, then I include them in the param_grid

In [28]:
model = KerasClassifier(build_fn=create_model, verbose=1, epochs=1, batch_size=32)

**GridSearchCV**

In [32]:
# param_grid = {"dropout": [0.3, 0.4, 0.5]}

In [35]:
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, return_train_score=True, cv=2)

In [36]:
# grid_result = grid.fit(X_train, y_train)

Epoch 1/1
12500/12500 [==============================] - 16s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 16s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 16s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 16s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 15s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 16s 1ms/step
Epoch 1/1
25000/25000 [==============================] - 165s 7ms/step - loss: 0.4235 - acc: 0.7890


Now, After the grid search we can view the following:

In [37]:
# # Best score
# grid_result.best_score_

0.84260000001907354

In [38]:
# # Best params
# grid_result.best_params_

{'dropout': 0.3}

cv_results_ include a lot of information about the run. It can be viewed directly (print(grid_result.cv_results_)), or be be imported into a pandas DataFrame.

In [39]:
# grid_result.cv_results_

{'mean_fit_time': array([ 82.74873328,  83.85946524,  84.53690612]),
 'mean_score_time': array([ 16.6534431 ,  16.56416392,  16.52149272]),
 'mean_test_score': array([ 0.8426 ,  0.82388,  0.81444]),
 'mean_train_score': array([ 0.87356,  0.84876,  0.82976]),
 'param_dropout': masked_array(data = [0.3 0.4 0.5],
              mask = [False False False],
        fill_value = ?),
 'params': [{'dropout': 0.3}, {'dropout': 0.4}, {'dropout': 0.5}],
 'rank_test_score': array([1, 2, 3]),
 'split0_test_score': array([ 0.84472,  0.81864,  0.81752]),
 'split0_train_score': array([ 0.88008,  0.84488,  0.83592]),
 'split1_test_score': array([ 0.84048,  0.82912,  0.81136]),
 'split1_train_score': array([ 0.86704,  0.85264,  0.8236 ]),
 'std_fit_time': array([ 0.49385214,  0.09287059,  0.99728048]),
 'std_score_time': array([ 0.28842354,  0.09349632,  0.08408713]),
 'std_test_score': array([ 0.00212,  0.00524,  0.00308]),
 'std_train_score': array([ 0.00652,  0.00388,  0.00616])}

In [40]:
# pd.DataFrame(grid_result.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_dropout,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,82.748733,16.653443,0.84260,0.87356,0.3,{'dropout': 0.3},1,0.84472,0.88008,0.84048,0.86704,0.493852,0.288424,0.00212,0.00652
1,83.859465,16.564164,0.82388,0.84876,0.4,{'dropout': 0.4},2,0.81864,0.84488,0.82912,0.85264,0.092871,0.093496,0.00524,0.00388
2,84.536906,16.521493,0.81444,0.82976,0.5,{'dropout': 0.5},3,0.81752,0.83592,0.81136,0.82360,0.997280,0.084087,0.00308,0.00616


**How to print a summary of the model?**
Just use the summary() function to the create_model() function

In [41]:
create_model().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 400, 50)           250000    
_________________________________________________________________
spatial_dropout1d_28 (Spatia (None, 400, 50)           0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_28 (Glo (None, 250)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 250)               62750     
_________________________________________________________________
dropout_28 (Dropout)         (None, 250)               0         
_________________________________________________________________
activation_55 (Activation)   (None, 250)               0         
__________

**Randomized search**

If a float - use "uniform"().
if an integer - use sp_randint().
for categorical it is like in grid search - ["one", "two]

In [15]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
param_dist= {"dropout": uniform(0.3, 0.5)}
random_grid = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter = 3, n_jobs=1, return_train_score=True, cv=2)
random_grid_result = random_grid.fit(X_train, y_train)

Epoch 1/1
12500/12500 [==============================] - 15s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 15s 1ms/step
Epoch 1/1
12500/12500 [==============================] - 12s 950us/step
Epoch 1/1
12500/12500 [==============================] - 12s 943us/step
Epoch 1/1
12500/12500 [==============================] - 11s 909us/step
Epoch 1/1
12500/12500 [==============================] - 11s 920us/step
Epoch 1/1
25000/25000 [==============================] - 108s 4ms/step - loss: 0.4291 - acc: 0.7797


In [16]:
random_grid_result.best_score_

0.84960000000000002

In [17]:
random_grid_result.best_params_

{'dropout': 0.30397377632925776}

In [18]:
pd.DataFrame(random_grid_result.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_dropout,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,81.198032,15.639521,0.55556,0.55788,0.761611,{'dropout': 0.761611244548},3,0.61576,0.61112,0.49536,0.50464,0.388168,0.135061,0.06020,0.05324
1,62.265965,12.302037,0.84960,0.88104,0.303974,{'dropout': 0.303973776329},1,0.85608,0.88968,0.84312,0.87240,6.730346,0.716967,0.00648,0.00864
2,54.754967,11.811988,0.56836,0.56644,0.79166,{'dropout': 0.791659673494},2,0.63208,0.63752,0.50464,0.49536,0.026236,0.233368,0.06372,0.07108
